In [1]:
import os
import pandas as pd
import json
import pprint
import re
import nbformat

In [194]:
# Define the path
#path = os.path.join("GitHub", "DataProcessing")

# Print the path
#print(path)

In [195]:
#pwd

In [196]:
#td = pd.read_json("..\DataProcessing\TestData\message.json", orient='index')

In [197]:
# Read the JSON file
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)

In [198]:
# Access the 'participants' array within 'metadata'
participantsData = data['metadata']['participants']
participantsData

['NLZhdGCfh-EgdvaODKW2IlCFzJBMtE4Dr9SFgKNpE1UC6GDNH2u2TQmGZoqSLXcVjo-erCaRQVV0cQ',
 'vfktkyVovXtnWWgFEDKxdtfJFVHSV3tAJeoT_UjbZB8TlE8tyVYLyC23nudviaAwk0c6AkFGGTYJhQ',
 'O9jmQXkdmuplQq2s4YGCd-x2VnJetyQ7ssrBczKK-xwT0PYaVpMyOWIHX_iIIznvHmk0M5_uE_CRzw',
 'HHVQBTfTWCQzCredc0AtS4BotTQqhQhK_7T4241WyfhlVNI1Oy7SHDjpPOg4T0KOuHu7eIS8gsRXsA',
 '-e-lfG5zpkNcr96I_Q_5ZdDefVsot-QfGnTH2QSHv-cMyP1_u8NUf-hImv_nQKwCUIq05DF8yFAIEw',
 'tTu5MniQjpI3G_eEm31jSm8ZhwCYJjRUQgQu0ynIs05DxKEkiRMdpOBq_3axazOPR4eP-M_gYcsHxg',
 '8s-H0x0EL9H84mDtglkoIYtyxASxccITkQaICj8RKUore3nABvXCZg_apnRSXMwLJYd2nCDUkJVk_A',
 'JBQKhP0EHYxtzSCrglHYSWq8of63qVowOOQ9NyMm6Rfv_Ups7-qrn11AgSY9-RoKro32bTDMsAqF3A',
 'y_0f-NLP6I974-LIJDU7WDu4qdHkqjVDaOUODjo6U85Jc50Z31pyAxvqKbfd4ceS5GCVWT-h8O8-0Q',
 'bztkNWyCR__do5ACtWbe4vhBdR1eXLFwxDdql2Y9kfT26TbuiNLgxNhLwTpoJWToU5UK-FCXFGTBWA']

In [199]:
puuidPair = data['info']['participants']
puuidPair

[{'participantId': 1,
  'puuid': 'NLZhdGCfh-EgdvaODKW2IlCFzJBMtE4Dr9SFgKNpE1UC6GDNH2u2TQmGZoqSLXcVjo-erCaRQVV0cQ'},
 {'participantId': 2,
  'puuid': 'vfktkyVovXtnWWgFEDKxdtfJFVHSV3tAJeoT_UjbZB8TlE8tyVYLyC23nudviaAwk0c6AkFGGTYJhQ'},
 {'participantId': 3,
  'puuid': 'O9jmQXkdmuplQq2s4YGCd-x2VnJetyQ7ssrBczKK-xwT0PYaVpMyOWIHX_iIIznvHmk0M5_uE_CRzw'},
 {'participantId': 4,
  'puuid': 'HHVQBTfTWCQzCredc0AtS4BotTQqhQhK_7T4241WyfhlVNI1Oy7SHDjpPOg4T0KOuHu7eIS8gsRXsA'},
 {'participantId': 5,
  'puuid': '-e-lfG5zpkNcr96I_Q_5ZdDefVsot-QfGnTH2QSHv-cMyP1_u8NUf-hImv_nQKwCUIq05DF8yFAIEw'},
 {'participantId': 6,
  'puuid': 'tTu5MniQjpI3G_eEm31jSm8ZhwCYJjRUQgQu0ynIs05DxKEkiRMdpOBq_3axazOPR4eP-M_gYcsHxg'},
 {'participantId': 7,
  'puuid': '8s-H0x0EL9H84mDtglkoIYtyxASxccITkQaICj8RKUore3nABvXCZg_apnRSXMwLJYd2nCDUkJVk_A'},
 {'participantId': 8,
  'puuid': 'JBQKhP0EHYxtzSCrglHYSWq8of63qVowOOQ9NyMm6Rfv_Ups7-qrn11AgSY9-RoKro32bTDMsAqF3A'},
 {'participantId': 9,
  'puuid': 'y_0f-NLP6I974-LIJDU7WDu4qdHkqjVDaOUODj

In [201]:
# participantData[]=null
participantData = []

participant_champion_mapping = {
    1: "tmp1",
    2: "tmp2",
    3: "tmp3",
    4: "tmp4",
    5: "tmp5",
    6: "tmp6",
    7: "tmp7",
    8: "tmp8",
    9: "tmp9",
    10: "tmp10"
}

def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = participant_champion_mapping.get(participantId, "Unknown Champion")
            # championName = participant['championName']
            addParticipantData(participantId, puuid, championName)

# Example usage
findParticipantData(data)

print("Participant Data:")
for participant in participantData:
    print(f"Participant ID: {participant[0]}, PUUID: {participant[1]}, Champion Name: {participant[2]}")


Participant Data:
Participant ID: 1, PUUID: NLZhdGCfh-EgdvaODKW2IlCFzJBMtE4Dr9SFgKNpE1UC6GDNH2u2TQmGZoqSLXcVjo-erCaRQVV0cQ, Champion Name: tmp1
Participant ID: 2, PUUID: vfktkyVovXtnWWgFEDKxdtfJFVHSV3tAJeoT_UjbZB8TlE8tyVYLyC23nudviaAwk0c6AkFGGTYJhQ, Champion Name: tmp2
Participant ID: 3, PUUID: O9jmQXkdmuplQq2s4YGCd-x2VnJetyQ7ssrBczKK-xwT0PYaVpMyOWIHX_iIIznvHmk0M5_uE_CRzw, Champion Name: tmp3
Participant ID: 4, PUUID: HHVQBTfTWCQzCredc0AtS4BotTQqhQhK_7T4241WyfhlVNI1Oy7SHDjpPOg4T0KOuHu7eIS8gsRXsA, Champion Name: tmp4
Participant ID: 5, PUUID: -e-lfG5zpkNcr96I_Q_5ZdDefVsot-QfGnTH2QSHv-cMyP1_u8NUf-hImv_nQKwCUIq05DF8yFAIEw, Champion Name: tmp5
Participant ID: 6, PUUID: tTu5MniQjpI3G_eEm31jSm8ZhwCYJjRUQgQu0ynIs05DxKEkiRMdpOBq_3axazOPR4eP-M_gYcsHxg, Champion Name: tmp6
Participant ID: 7, PUUID: 8s-H0x0EL9H84mDtglkoIYtyxASxccITkQaICj8RKUore3nABvXCZg_apnRSXMwLJYd2nCDUkJVk_A, Champion Name: tmp7
Participant ID: 8, PUUID: JBQKhP0EHYxtzSCrglHYSWq8of63qVowOOQ9NyMm6Rfv_Ups7-qrn11AgSY9-RoKro32bTDMsA

In [8]:
# Get every timestamp within a game paired with corresponding minute
for counter, frame in enumerate(data['info']['frames']):
    timestamp = frame['timestamp']
    print(f"{counter + 1} Timestamp = {timestamp}")

1 Timestamp = 0
2 Timestamp = 60016
3 Timestamp = 120021
4 Timestamp = 180035
5 Timestamp = 240035
6 Timestamp = 300066
7 Timestamp = 360069
8 Timestamp = 420084
9 Timestamp = 480090
10 Timestamp = 540109
11 Timestamp = 600126
12 Timestamp = 660130
13 Timestamp = 720141
14 Timestamp = 780169
15 Timestamp = 840170
16 Timestamp = 900173
17 Timestamp = 960186
18 Timestamp = 1020272
19 Timestamp = 1080286
20 Timestamp = 1140318
21 Timestamp = 1200328
22 Timestamp = 1260354
23 Timestamp = 1320366
24 Timestamp = 1380386
25 Timestamp = 1440411
26 Timestamp = 1500414
27 Timestamp = 1560433
28 Timestamp = 1620447
29 Timestamp = 1680454
30 Timestamp = 1740480
31 Timestamp = 1800496
32 Timestamp = 1860523
33 Timestamp = 1892560


In [19]:
frames = data['info']['frames']
participantFramesList = [frame.get('participantFrames', {}) for frame in frames]

In [82]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value

    return excludedData


In [83]:
def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key}:")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")


In [87]:
def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = "2"
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


Timestamp at minute 2: 120021
championStats:
    armor: 33
    attackDamage: 67
    attackSpeed: 114
    health: 520
    healthMax: 684
    healthRegen: 7
    magicResist: 33
    movementSpeed: 340
  currentGold: 199
damageStats:
    totalDamageDone: 3801
    totalDamageTaken: 709
  jungleMinionsKilled: 5
  level: 2
  participantId: 2
position:
    x: 8451
    y: 2647
  totalGold: 649
  xp: 367

---



In [121]:
# [NOTFINISHED] Get killed within specified timestamps 
def getChampionKilled(data, start_timestamp, end_timestamp):
    for frame in data['info']['frames']:
        timestamp = frame['timestamp']
        
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                kill_timestamp = event['timestamp']
                
                if start_timestamp <= kill_timestamp <= end_timestamp:
                    print(f"Timestamp of CHAMPION_KILL event: {kill_timestamp}")
                    print("Victim Damage Dealt:")
                    
                    # Assuming 'victimDamageDealt' is a list in the 'CHAMPION_KILL' event
                    for damage_info in event.get('victimDamageDealt', []):
                        print(f"  Name: {damage_info['name']}")
                        print(f"  ID: {damage_info['participantId']}")
                        
                    for damage_info in event.get('victimDamageReceived', []):
                        #if event['type'] == 'OTHER':
                        print(f"  Name: {damage_info['name']}")
                    
                    #print(f"  ID: {damage_info['victimId']}")

# Example usage: Specify the start and end timestamps
start_timestamp = 120000  # Replace with your desired start timestamp
end_timestamp = 150000    # Replace with your desired end timestamp

# Call the function with the specified time frame
getChampionKilled(data, start_timestamp, end_timestamp)

Timestamp of CHAMPION_KILL event: 134226
Victim Damage Dealt:
  Name: Sion
  ID: 1
  Name: Sion
  ID: 1
  Name: Sion
  ID: 1
  Name: Sion
  ID: 1
  Name: Sion
  ID: 1
  Name: SRU_OrderMinionMelee
  Name: SRU_OrderMinionMelee
  Name: SRU_OrderMinionMelee
  Name: Volibear
  Name: Volibear
  Name: Volibear
  Name: Volibear
  Name: Volibear


In [105]:
# [NOTFINSHED] Print out all champion kill information at specific kill timestamp
def get_event_of_champion_kill_at_timestamp(data, target_timestamp):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL' and event['timestamp'] == target_timestamp:
                return event

# Example: Get CHAMPION_KILL event data at timestamp 134226
target_timestamp = 134226
champion_kill_event = get_event_of_champion_kill_at_timestamp(data, target_timestamp)

# Print or use the champion_kill_event data as needed
if champion_kill_event:
    print(f"CHAMPION_KILL event data:")
    pprint.pprint(champion_kill_event)
else:
    print(f"No CHAMPION_KILL event found at timestamp {target_timestamp}.")


CHAMPION_KILL event data:
{'bounty': 400,
 'killStreakLength': 0,
 'killerId': 1,
 'position': {'x': 3958, 'y': 13447},
 'shutdownBounty': 0,
 'timestamp': 134226,
 'type': 'CHAMPION_KILL',
 'victimDamageDealt': [{'basic': False,
                        'magicDamage': 0,
                        'name': 'Sion',
                        'participantId': 1,
                        'physicalDamage': 233,
                        'spellName': 'sionq',
                        'spellSlot': 0,
                        'trueDamage': 0,
                        'type': 'OTHER'},
                       {'basic': True,
                        'magicDamage': 0,
                        'name': 'Sion',
                        'participantId': 1,
                        'physicalDamage': 105,
                        'spellName': 'sionbasicattack3',
                        'spellSlot': 66,
                        'trueDamage': 0,
                        'type': 'OTHER'},
                       {'basic': Fa

In [106]:
import json

# Load JSON data
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)

# Extract relevant data from JSON
frames_data = data['info']['frames']
participants_data = [frame['participantFrames'] for frame in frames_data]

# Define participant IDs
participant_ids = ['1', '2']

# Display the extracted parameter values
for participant_id in participant_ids:
    print(f"Participant {participant_id} Parameters:")
    for idx, frame_data in enumerate(participants_data):
        damage_stats = frame_data.get(participant_id, {}).get('damageStats', {})
        total_damage_done = damage_stats.get('totalDamageDone', 0)
        total_damage_done_to_champions = damage_stats.get('totalDamageDoneToChampions', 0)
        total_damage_taken = damage_stats.get('totalDamageTaken', 0)
        
        print(f"Frame {idx + 1}:")
        print(f"  Total Damage Done: {total_damage_done}")
        print(f"  Total Damage Done To Champions: {total_damage_done_to_champions}")
        print(f"  Total Damage Taken: {total_damage_taken}")


Participant 1 Parameters:
Frame 1:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 2:
  Total Damage Done: 0
  Total Damage Done To Champions: 0
  Total Damage Taken: 0
Frame 3:
  Total Damage Done: 544
  Total Damage Done To Champions: 105
  Total Damage Taken: 291
Frame 4:
  Total Damage Done: 4071
  Total Damage Done To Champions: 1053
  Total Damage Taken: 774
Frame 5:
  Total Damage Done: 5753
  Total Damage Done To Champions: 1195
  Total Damage Taken: 1015
Frame 6:
  Total Damage Done: 11940
  Total Damage Done To Champions: 2301
  Total Damage Taken: 1937
Frame 7:
  Total Damage Done: 12122
  Total Damage Done To Champions: 2301
  Total Damage Taken: 1968
Frame 8:
  Total Damage Done: 20977
  Total Damage Done To Champions: 3833
  Total Damage Taken: 3208
Frame 9:
  Total Damage Done: 23368
  Total Damage Done To Champions: 4180
  Total Damage Taken: 3677
Frame 10:
  Total Damage Done: 33392
  Total Damage Done To Champions: 5733
  Total

In [162]:
def getTimestampOfChampionKillWithId(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                killerId = event.get('killerId')
                victimId = event.get('victimId')
                victimNames = [victim.get('name') for victim in event.get('victimDamageDealt', [])]
                killerNames = [killer.get('name') for killer in event.get('victimDamageReceived', [])]

                if killerNames:  # Check if there are killers
                    # Use the last killer as the one who landed the final hit
                    last_killer = killerNames[-1]
                    print(f"{victimNames[0]} was killed with ID: {victimId}")
                    print(f"Timestamp of CHAMPION_KILL: {timestamp}")
                    print(f"Killed by {last_killer} with ID: {killerId}\n")

getTimestampOfChampionKillWithId(data)


Sion was killed with ID: 6
Timestamp of CHAMPION_KILL: 134226
Killed by Volibear with ID: 1

Brand was killed with ID: 5
Timestamp of CHAMPION_KILL: 171327
Killed by Jhin with ID: 9

Kaisa was killed with ID: 4
Timestamp of CHAMPION_KILL: 187450
Killed by Jhin with ID: 9

Veigar was killed with ID: 8
Timestamp of CHAMPION_KILL: 192190
Killed by Hwei with ID: 3

Hwei was killed with ID: 3
Timestamp of CHAMPION_KILL: 209499
Killed by Kayn with ID: 7

Kayn was killed with ID: 7
Timestamp of CHAMPION_KILL: 255814
Killed by Volibear with ID: 2

Sion was killed with ID: 6
Timestamp of CHAMPION_KILL: 260683
Killed by Briar with ID: 1

Nautilus was killed with ID: 10
Timestamp of CHAMPION_KILL: 303299
Killed by SRU_OrderMinionRanged with ID: 4

Kayn was killed with ID: 7
Timestamp of CHAMPION_KILL: 352366
Killed by Briar with ID: 2

Sion was killed with ID: 6
Timestamp of CHAMPION_KILL: 389728
Killed by SRU_OrderMinionRanged with ID: 1

Veigar was killed with ID: 8
Timestamp of CHAMPION_KILL: 

IndexError: list index out of range

In [177]:
def createParticipantIdToNameMapping(data):
    participant_mapping = {}

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                killerId = event.get('killerId')
                killerName = event.get('killerName')
                victimId = event.get('victimId')
                victimName = event.get('victimName')

                if killer_id and killer_name:
                    participant_mapping[killer_id] = killer_name
                if victim_id and victim_name:
                    participant_mapping[victim_id] = victim_name

    return participant_mapping

def findParticipantName(participantId, participantMapping):
    return participantMapping.get(participantId, f"Unknown participant with ID: {participantId}")


participantMapping = createParticipantIdToNameMapping(data)

# Replace the participantId with the ID you want to search for
desiredParticipantId = 7
participantName = findParticipantName(desiredParticipantId, participantMapping)

print(f"Participant with ID {desiredParticipantId} is: {participantName}")


Participant with ID 7 is: Unknown participant with ID: 7


In [167]:
def getTimestampOfChampionKillWithId(data):
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'CHAMPION_KILL':
                timestamp = event['timestamp']
                killerId = event.get('killerId')
                victimId = event.get('victimId')
                victimNames = [victim.get('name') for victim in event.get('victimDamageDealt', [])]
                killerNames = [killer.get('name') for killer in event.get('victimDamageReceived', [])]

                if victimNames:  # Check if there are victim names
                    victim_name = victimNames[0]
                    print(f"{victim_name} was killed with ID: {victimId}")
                else:
                    
                    print(f"{victimId}")

                if killerNames:  # Check if there are killers
                    # Use the last killer as the one who landed the final hit
                    last_killer = killerNames[-1]
                    print(f"Killed by {last_killer} with ID: {killerId}")
                    print(f"Timestamp of CHAMPION_KILL: {timestamp}\n")


getTimestampOfChampionKillWithId(data)


Sion was killed with ID: 6
Killed by Volibear with ID: 1
Timestamp of CHAMPION_KILL: 134226

Brand was killed with ID: 5
Killed by Jhin with ID: 9
Timestamp of CHAMPION_KILL: 171327

Kaisa was killed with ID: 4
Killed by Jhin with ID: 9
Timestamp of CHAMPION_KILL: 187450

Veigar was killed with ID: 8
Killed by Hwei with ID: 3
Timestamp of CHAMPION_KILL: 192190

Hwei was killed with ID: 3
Killed by Kayn with ID: 7
Timestamp of CHAMPION_KILL: 209499

Kayn was killed with ID: 7
Killed by Volibear with ID: 2
Timestamp of CHAMPION_KILL: 255814

Sion was killed with ID: 6
Killed by Briar with ID: 1
Timestamp of CHAMPION_KILL: 260683

Nautilus was killed with ID: 10
Killed by SRU_OrderMinionRanged with ID: 4
Timestamp of CHAMPION_KILL: 303299

Kayn was killed with ID: 7
Killed by Briar with ID: 2
Timestamp of CHAMPION_KILL: 352366

Sion was killed with ID: 6
Killed by SRU_OrderMinionRanged with ID: 1
Timestamp of CHAMPION_KILL: 389728

Veigar was killed with ID: 8
Killed by Hwei with ID: 2
Ti

In [10]:
def getParticipantIdByKill(data):
    for frame in data['info']['frames']:
        for event in frame.get('events',[]):
            eventType = event.get('type')
            if eventType == 'CHAMPION_KILL':
                killerId = event.get('killerId')
                print(f"Participant id: {killerId}") 

getParticipantIdByKill(data)

Participant id: 1
Participant id: 9
Participant id: 9
Participant id: 3
Participant id: 7
Participant id: 2
Participant id: 1
Participant id: 4
Participant id: 2
Participant id: 1
Participant id: 2
Participant id: 6
Participant id: 10
Participant id: 7
Participant id: 2
Participant id: 7
Participant id: 8
Participant id: 2
Participant id: 1
Participant id: 2
Participant id: 7
Participant id: 7
Participant id: 10
Participant id: 8
Participant id: 4
Participant id: 4
Participant id: 8
Participant id: 7
Participant id: 4
Participant id: 4
Participant id: 1
Participant id: 6
Participant id: 6
Participant id: 4
Participant id: 9
Participant id: 1
Participant id: 8
Participant id: 2
Participant id: 8
Participant id: 2
Participant id: 10
Participant id: 7
Participant id: 7
Participant id: 1
Participant id: 9
Participant id: 5
Participant id: 7
Participant id: 6
Participant id: 9
Participant id: 9
Participant id: 8
Participant id: 3
Participant id: 1
Participant id: 1
Participant id: 1
Partici

In [15]:
import json

# Load JSON data
with open("../DataProcessing/TestData/message.json", "r") as file:
    data = json.load(file)

# Extract relevant data from JSON
frames_data = data['info']['frames']
participants_data = [frame['participantFrames'] for frame in frames_data]

# Define participant ID and time frames to subtract
participant_id_to_subtract = '1'
start_frame = 3  # Adjust as needed
end_frame = 4    # Adjust as needed

# Initialize variables to store total damage done to champions
total_damage_to_champions_start = 0
total_damage_to_champions_end = 0

# Display the extracted parameter values and calculate total damage done to champions
for idx, frame_data in enumerate(participants_data):
    damage_stats = frame_data.get(participant_id_to_subtract, {}).get('damageStats', {})
    total_damage_done_to_champions = damage_stats.get('totalDamageDoneToChampions', 0)

    if idx + 1 == start_frame:
        total_damage_to_champions_start = total_damage_done_to_champions
    elif idx + 1 == end_frame:
        total_damage_to_champions_end = total_damage_done_to_champions

# Print the total damage done to champions only for the specified frames
print(f"PRE")
print(f"Frame {start_frame}:")
print(f"  Total Damage Done To Champions: {total_damage_to_champions_start}")
print(f"POST")
print(f"Frame {end_frame}:")
print(f"  Total Damage Done To Champions: {total_damage_to_champions_end}")

# Calculate and print the difference
difference = total_damage_to_champions_end - total_damage_to_champions_start
print(f"\nTotal Damage Done To Champions Difference: {difference}")



PRE
Frame 3:
  Total Damage Done To Champions: 105
POST
Frame 4:
  Total Damage Done To Champions: 1053

Total Damage Done To Champions Difference: 948


In [18]:
def calculate_damage_difference(data, participant_id, start_frame, end_frame):
    # Extract relevant data from pre-loaded JSON
    frames_data = data['info']['frames']
    participants_data = [frame['participantFrames'] for frame in frames_data]

    # Initialize variables to store total damage done to champions
    total_damage_to_champions_start = 0
    total_damage_to_champions_end = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frame_data in enumerate(participants_data):
        damage_stats = frame_data.get(participant_id, {}).get('damageStats', {})
        total_damage_done_to_champions = damage_stats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == start_frame:
            total_damage_to_champions_start = total_damage_done_to_champions
        elif idx + 1 == end_frame:
            total_damage_to_champions_end = total_damage_done_to_champions

    # Print the total damage done to champions only for the specified frames
    print(f"PRE")
    print(f"Frame {start_frame}:")
    print(f"  Total Damage Done To Champions: {total_damage_to_champions_start}")
    print(f"POST")
    print(f"Frame {end_frame}:")
    print(f"  Total Damage Done To Champions: {total_damage_to_champions_end}")

    # Calculate and print the difference
    difference = total_damage_to_champions_end - total_damage_to_champions_start
    print(f"\nTotal Damage Done To Champions Difference: {difference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculate_damage_difference(data, '1', 7, 8)


PRE
Frame 7:
  Total Damage Done To Champions: 0
POST
Frame 8:
  Total Damage Done To Champions: 0

Total Damage Done To Champions Difference: 0


In [23]:
def calculate_damage_difference(data, participant_id, start_frame, end_frame):
    # Extract relevant data from pre-loaded JSON
    frames_data = data['info']['frames']
    participants_data = [frame['participantFrames'] for frame in frames_data]

    # Determine the extra participant_id based on the given participant_id
    extra_participant_id = str((int(participant_id) + 5) % 10) if int(participant_id) < 6 else str((int(participant_id) - 5) % 10)

    # Initialize variables to store total damage done to champions
    total_damage_to_champions_start = 0
    total_damage_to_champions_end = 0
    total_damage_to_champions_extra_start = 0
    total_damage_to_champions_extra_end = 0

    # Display the extracted parameter values and calculate total damage done to champions
    for idx, frame_data in enumerate(participants_data):
        damage_stats = frame_data.get(participant_id, {}).get('damageStats', {})
        total_damage_done_to_champions = damage_stats.get('totalDamageDoneToChampions', 0)

        extra_damage_stats = frame_data.get(extra_participant_id, {}).get('damageStats', {})
        total_damage_done_to_champions_extra = extra_damage_stats.get('totalDamageDoneToChampions', 0)

        if idx + 1 == start_frame:
            total_damage_to_champions_start = total_damage_done_to_champions
            total_damage_to_champions_extra_start = total_damage_done_to_champions_extra
        elif idx + 1 == end_frame:
            total_damage_to_champions_end = total_damage_done_to_champions
            total_damage_to_champions_extra_end = total_damage_done_to_champions_extra

    # Print the total damage done to champions for the participant and extra participant
    print(f"PRE")
    print(f"Frame {start_frame} - Participant {participant_id}:")
    print(f"  Total Damage Done To Champions: {total_damage_to_champions_start}")
    print(f"Frame {start_frame} - Extra Participant {extra_participant_id}:")
    print(f"  Total Damage Done To Champions: {total_damage_to_champions_extra_start}")
    
    print(f"POST")
    print(f"Frame {end_frame} - Participant {participant_id}:")
    print(f"  Total Damage Done To Champions: {total_damage_to_champions_end}")
    print(f"Frame {end_frame} - Extra Participant {extra_participant_id}:")
    print(f"  Total Damage Done To Champions: {total_damage_to_champions_extra_end}")

    # Calculate and print the differences
    difference = total_damage_to_champions_end - total_damage_to_champions_start
    extra_difference = total_damage_to_champions_extra_end - total_damage_to_champions_extra_start
    print(f"\nTotal Damage Done To Champions Difference (Participant {participant_id}): {difference}")
    print(f"Total Damage Done To Champions Difference (Extra Participant {extra_participant_id}): {extra_difference}")

# Example usage:
# Assuming 'data' is the loaded JSON data
calculate_damage_difference(data, '6', 4, 5)


PRE
Frame 4 - Participant 6:
  Total Damage Done To Champions: 420
Frame 4 - Extra Participant 1:
  Total Damage Done To Champions: 1053
POST
Frame 5 - Participant 6:
  Total Damage Done To Champions: 605
Frame 5 - Extra Participant 1:
  Total Damage Done To Champions: 1195

Total Damage Done To Champions Difference (Participant 6): 185
Total Damage Done To Champions Difference (Extra Participant 1): 142


In [52]:
def calculate_damage_difference(data, participant_id, start_frame, end_frame):
    # Extract relevant data from pre-loaded JSON
    frames_data = data['info']['frames']
    participants_data = [frame['participantFrames'] for frame in frames_data]

    # Determine the extra participant_id based on the given participant_id
    extra_participant_id = str((int(participant_id) + 5) % 10) if int(participant_id) < 6 else str((int(participant_id) - 5) % 10)

    # Initialize variables to store total damage done parameters
    total_damage_done_start = 0
    total_damage_done_end = 0
    total_damage_done_extra_start = 0
    total_damage_done_extra_end = 0

    total_damage_done_to_champions_start = 0
    total_damage_done_to_champions_end = 0
    total_damage_done_to_champions_extra_start = 0
    total_damage_done_to_champions_extra_end = 0

    total_damage_taken_start = 0
    total_damage_taken_end = 0
    total_damage_taken_extra_start = 0
    total_damage_taken_extra_end = 0
    
    total_damage_taken_compared=0
    total_damage_done_compared=0
    total_damage_done_to_champions_compared=0

    # Display the extracted parameter values and calculate total damage done parameters
    for idx, frame_data in enumerate(participants_data):
        damage_stats = frame_data.get(participant_id, {}).get('damageStats', {})
        total_damage_done = damage_stats.get('totalDamageDone', 0)
        total_damage_done_to_champions = damage_stats.get('totalDamageDoneToChampions', 0)
        total_damage_taken = damage_stats.get('totalDamageTaken', 0)

        extra_damage_stats = frame_data.get(extra_participant_id, {}).get('damageStats', {})
        total_damage_done_extra = extra_damage_stats.get('totalDamageDone', 0)
        total_damage_done_to_champions_extra = extra_damage_stats.get('totalDamageDoneToChampions', 0)
        total_damage_taken_extra = extra_damage_stats.get('totalDamageTaken', 0)

        if idx + 1 == start_frame:
            total_damage_done_start = total_damage_done
            total_damage_done_to_champions_start = total_damage_done_to_champions
            total_damage_taken_start = total_damage_taken
            total_damage_done_extra_start = total_damage_done_extra
            total_damage_done_to_champions_extra_start = total_damage_done_to_champions_extra
            total_damage_taken_extra_start = total_damage_taken_extra
        elif idx + 1 <= end_frame:
            total_damage_done_end = total_damage_done
            total_damage_done_to_champions_end = total_damage_done_to_champions
            total_damage_taken_end = total_damage_taken
            total_damage_done_extra_end = total_damage_done_extra
            total_damage_done_to_champions_extra_end = total_damage_done_to_champions_extra
            total_damage_taken_extra_end = total_damage_taken_extra
    
    
    
    # Print the pre and post data for the first participant
    print(f"Participant {participant_id}:")
    print(f"  Frame {start_frame} - Total Damage Done: {total_damage_done_start}")
    print(f"  Frame {end_frame} - Total Damage Done: {total_damage_done_end}")
    print(f"  Frame {start_frame} - Total Damage Done To Champions: {total_damage_done_to_champions_start}")
    print(f"  Frame {end_frame} - Total Damage Done To Champions: {total_damage_done_to_champions_end}")
    print(f"  Frame {start_frame} - Total Damage Taken: {total_damage_taken_start}")
    print(f"  Frame {end_frame} - Total Damage Taken: {total_damage_taken_end}")

    # Print the data for the extra participant
    print(f"\nEnemy Laner:")
    print(f"  Frame {start_frame} - Total Damage Done: {total_damage_done_extra_start}")
    print(f"  Frame {end_frame} - Total Damage Done: {total_damage_done_extra_end}")
    print(f"  Frame {start_frame} - Total Damage Done To Champions: {total_damage_done_to_champions_extra_start}")
    print(f"  Frame {end_frame} - Total Damage Done To Champions: {total_damage_done_to_champions_extra_end}")
    print(f"  Frame {start_frame} - Total Damage Taken: {total_damage_taken_extra_start}")
    print(f"  Frame {end_frame} - Total Damage Taken: {total_damage_taken_extra_end}")

    # Calculate and print the differences
    damage_done_difference = total_damage_done_end - total_damage_done_start
    extra_damage_done_difference = total_damage_done_extra_end - total_damage_done_extra_start
    damage_done_to_champions_difference = total_damage_done_to_champions_end - total_damage_done_to_champions_start
    extra_damage_done_to_champions_difference = total_damage_done_to_champions_extra_end - total_damage_done_to_champions_extra_start
    damage_taken_difference = total_damage_taken_end - total_damage_taken_start
    extra_damage_taken_difference = total_damage_taken_extra_end - total_damage_taken_extra_start

    print(f"\nTotal Damage Done Difference (Participant {participant_id}): {damage_done_difference}")
    print(f"Total Damage Done Difference (Enemy Laner): {extra_damage_done_difference}")
    print(f"Total Damage Done To Champions Difference (Participant {participant_id}): {damage_done_to_champions_difference}")
    print(f"Total Damage Done To Champions Difference (Enemy Laner): {extra_damage_done_to_champions_difference}")
    print(f"Total Damage Taken Difference (Participant {participant_id}): {damage_taken_difference}")
    print(f"Total Damage Taken Difference (Enemy Laner): {extra_damage_taken_difference}")
    
    # Calculate differences between participant and enemy
    total_damage_taken_compared=damage_taken_difference - extra_damage_taken_difference
    total_damage_done_compared=damage_done_difference - extra_damage_done_difference
    total_damage_done_to_champions_compared=damage_done_to_champions_difference - extra_damage_done_to_champions_difference
    
    if total_damage_taken_compared >= 0:
        print(f"\nParticipant {participant_id} took {total_damage_taken_compared} more damage than enemy laner in given timeframe")
    else:
        print(f"\nParticipant {participant_id} took {abs(total_damage_taken_compared)} less damage than enemy laner in given timeframe")

    if total_damage_done_to_champions_compared >= 0:
        print(f"\nParticipant {participant_id} beat enemy laner in given timeframe with total damage done difference of: {total_damage_done_to_champions_compared}")
    else:
        print(f"\nParticipant {participant_id} was beat by enemy laner in given timeframe with total damage done difference of: {abs(total_damage_done_to_champions_compared)}")

    if total_damage_done_compared >= 0:
        print(f"\nParticipant {participant_id} beat enemy laner in given timeframe with total damage done difference of: {total_damage_done_compared}")
    else:
        print(f"\nParticipant {participant_id} was beat by enemy laner in given timeframe with total damage done difference of: {abs(total_damage_done_compared)}")
# Example usage:
# Assuming 'data' is the loaded JSON data
calculate_damage_difference(data, '6', 4, 6)


Participant 6:
  Frame 4 - Total Damage Done: 3198
  Frame 6 - Total Damage Done: 5736
  Frame 4 - Total Damage Done To Champions: 420
  Frame 6 - Total Damage Done To Champions: 870
  Frame 4 - Total Damage Taken: 1444
  Frame 6 - Total Damage Taken: 3223

Enemy Laner:
  Frame 4 - Total Damage Done: 4071
  Frame 6 - Total Damage Done: 11940
  Frame 4 - Total Damage Done To Champions: 1053
  Frame 6 - Total Damage Done To Champions: 2301
  Frame 4 - Total Damage Taken: 774
  Frame 6 - Total Damage Taken: 1937

Total Damage Done Difference (Participant 6): 2538
Total Damage Done Difference (Enemy Laner): 7869
Total Damage Done To Champions Difference (Participant 6): 450
Total Damage Done To Champions Difference (Enemy Laner): 1248
Total Damage Taken Difference (Participant 6): 1779
Total Damage Taken Difference (Enemy Laner): 1163

Participant 6 took 616 more damage than enemy laner in given timeframe

Participant 6 was beat by enemy laner in given timeframe with total damage done diff